# Week 2 Lab: Simple Data Lake with AWS Glue

In this lab, you will work with a simple data lake that uses Amazon S3 as its primary storage. The data lake bucket contains raw JSON files that you will transform using AWS Glue ETL, and then store the processed data in the same bucket. You will finally use AWS Glue crawler to populate the Glue data catalog with metadata about your processed data, and then use Amazon Athena to query your data using SQL statements. In the optional part of the lab, you will explore the effects of compression and partitioning when storing your data in S3.

If you get stuck while completing the coding exercises, you can download the solution files by running the following command in your Cloud9 environment:

```bash
aws s3 cp --recursive s3://dlai-data-engineering/labs/c3w2lab1-706170-solution/ ./
```

# Table of Contents

- [ 1 - Introduction](#1)
- [ 2 - Raw Data Exploration](#2)
  - [ Exercise 1](#ex01)
  - [ Exercise 2](#ex02)
- [ 3 - Raw Data Processing](#3)
  - [ Exercise 3](#ex03)
  - [ Exercise 4](#ex04)
- [ 4 - Transformation with AWS Glue ETL](#4)
  - [ 4.1 - Preparing the Scripts](#4.1)
    - [ Exercise 5](#ex05)
    - [ Exercise 6](#ex06)
  - [ 4.2 - Creating and Running the Glue Jobs](#4.2)
- [ 5 - Data Catalog with Glue Crawler](#5)
  - [ Exercise 7](#ex07)
- [ 6 - Data Querying with Athena](#6)
- [ 7 - Optional Experiments: Partitioning and Compression Features of Parquet Format](#7)
  - [ 7.1 - Experiments with the Glue Jobs](#7.1)
  - [ 7.2 - Experiment Results](#7.2)
- [ 8 - Clean up](#8)

First, import some relevant libraries for this lab:

In [ ]:
import datetime as dt
import gzip
import json
from typing import Dict

import awswrangler as wr
import boto3
import pandas as pd
import smart_open
import warnings

<a name='1'></a>
## 1 - Introduction

Assume you work as a Data Engineer at a retailer specializing in scale models of classic cars and other transportation. The data analysts are interested in conducting trend analysis for the top products reviewed in Amazon, to inform new product development. Recently, your team acquired Amazon toy review data and product info, and stored them in a data lake bucket. You are asked to clean the data and ensure its accessibility, so that the data analysts can retrieve the data with SQL-based queries. For the initial testing phase, the team has opted to use AWS Glue ETL for the initial data cleaning, and Amazon Athena to query the data. 

In this lab, you will use [Terraform](https://www.terraform.io/) to define the Glue ETL jobs. You will run these jobs to process the raw datasets and store the results as Parquet files in the same bucket. Using `boto3`, you will create a crawler that you will run over the processed data. The crawler will populate two catalog tables, each linked to a Parquet file. Finally you will query the data using Amazon Athena.

<img src="images/data_lake.png" width="600"/>

<a name='2'></a>
## 2 - Raw Data Exploration

The dataset consists of two JSON compressed files, one for reviews and one for metadata of the products. Here is an example of one review JSON object:

```json
{
  "reviewerID": "A2SUAM1J3GNN3B",
  "asin": "0000013714",
  "reviewerName": "J. McDonald",
  "helpful": [2, 3],
  "reviewText": "I bought this for my husband who plays the piano.  He is having a wonderful time playing these old hymns.  The music  is at times hard to read because we think the book was published for singing from more than playing from.  Great purchase though!",
  "overall": 5.0,
  "summary": "Heavenly Highway Hymns",
  "unixReviewTime": 1252800000,
  "reviewTime": "09 13, 2009"
}
```

Here is the description of the fields:

- `reviewerID` - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- `asin` - ID of the product, e.g. 0000013714
- `reviewerName` - name of the reviewer
- `helpful` - helpfulness rating of the review, e.g. 2/3
- `reviewText` - text of the review
- `overall` - rating of the product
- `summary` - summary of the review
- `unixReviewTime` - time of the review (unix time)
- `reviewTime` - time of the review (raw)

An here's an example of one product metadata JSON object:

```json
{
  "asin": "0641843224",
  "description": "Set your phasers to stun and prepare for a warp speed ride through the most memorable vocabulary from the sci-fi/fantasy genre.",
  "title": "McNeill Designs YBS Sci-fi/Fantasy Add-on Deck", 
  "price": 5.19,  
  "imUrl": "http://ecx.images-amazon.com/images/I/418t9AN9hiL._SY300_.jpg", 
  "related": 
  {
    "also_bought": ["B000EVLZ9U", "0641843208", "0641843216", "0641843267", "1450751210", "0641843232", "B00ALQFYGI", "B004G7B3NQ", "B002PDM288", "B009ZNJZV8", "B009YG928W", "B0063NC3N0"], 
    "also_viewed": ["B000EVLZ9U", "1450751210", "0641843208", "0641843267", "0641843232", "0641843216", "B003EIK136", "B004G7B3NQ", "B003N2Q5JC"], 
    "bought_together": ["B000EVLZ9U"]
  },
  "salesRank": {"Toys & Games": 154868}, 
  "brand": "McNeill Designs", 
  "categories": [["Toys & Games", "Games", "Card Games"]]
}
```

With the following fields:

- `asin` - ID of the product, e.g. 0000031852
- `description` - Description of the product
- `title` - name of the product
- `price` - price in US dollars (at time of crawl)
- `imUrl` - url of the product image
- `related` - related products (`also_bought`, `also_viewed`, `bought_together`)
- `salesRank` - sales rank information
- `brand` - brand name
- `categories` - list of categories the product belongs to

To get the name of the data lake bucket, go to the AWS console and click on the upper right part, where your username appears. Copy the **Account ID**. In the code below, set the variable `BUCKET_NAME` by replacing `<AWS-ACCOUNT-ID>` placeholder with the Account ID that you copied. The Account ID should contain only numbers without hyphens between them (e.g. 123412341234, not 1234-1234-1234 and the bucket name should have the same format as `de-c3w2lab1-123412341234-us-east-1-data-lake`).

In [ ]:
BUCKET_NAME = 'de-c3w2lab1-<AWS-ACCOUNT-ID>-us-east-1-data-lake'

The data lake bucket contains the raw JSON files. The bucket also contains samples of each dataset, which are of smaller size of the original data, that you will next interact with to explore the content of the data. The next cell consists of a function that you will use to load the data samples into a Pandas DataFrame so you can explore them. Run the following cell to define the function.

In [ ]:
def read_data_sample(bucket_name: str, s3_file_key: str) -> pd.DataFrame:
    """Reads review sample dataset

    Args:
        bucket_name (str): Bucket name
        s3_file_key (str): Dataset s3 key location

    Returns:
        pd.DataFrame: Read dataframe
    """
    s3_client = boto3.client('s3')
    source_uri = f's3://{bucket_name}/{s3_file_key}'
    json_list = []
    for json_line in smart_open.open(source_uri, transport_params={'client': s3_client}):
        json_list.append(json.loads(json_line))
    df = pd.DataFrame(json_list)
    return df


<a name='ex01'></a>
### Exercise 1

Complete the code to call the `read_data_sample()` function passing the `BUCKET_NAME` and the file key parameter as `'staging/reviews_Toys_and_Games_sample.json.gz'`. Then, take a look at the first 5 rows of this sample reviews dataset.

In [ ]:
### START CODE HERE ### (1 line of code)
review_sample_df = read_data_sample(bucket_name=BUCKET_NAME, s3_file_key='staging/reviews_Toys_and_Games_sample.json.gz') # @REPLACE EQUALS None(bucket_name=None, s3_file_key='None')
### END CODE HERE ###

review_sample_df.head(5)

Let's take a look at the data type of each of the columns of the dataset, and pay closer attention to the `unixReviewTime` and `helpful` columns.

In [ ]:
review_sample_df.dtypes

You can see that the `unixReviewTime` value is currently an integer that represents a Unix timestamp defined in terms of the number of seconds since January 1st, 1970 at UTC. Each `helpful` column consists of two numbers: the number of users who found the review helpful and the total number of users who rated the helpfulness of this review. Later in Exercise 3, you will transform this raw data to make it more useful for further analysis.

Let's check out the metadata dataset. 

<a name='ex02'></a>
### Exercise 2

Execute the `read_data_sample()` function with the same bucket name, but now set the file key parameter as `'staging/meta_Toys_and_Games_sample.json.gz'`. Then take a look at the first 5 rows of this sample metadata dataset.

In [ ]:
### START CODE HERE ### (1 line of code)
metadata_sample_df = read_data_sample(bucket_name=BUCKET_NAME, s3_file_key='staging/meta_Toys_and_Games_sample.json.gz') # @REPLACE EQUALS None(bucket_name=None, s3_file_key='None')
### END CODE HERE ###

metadata_sample_df.head(5)

Then, take a look at the column's data types.

In [ ]:
metadata_sample_df.dtypes

To make this data more useful for analysis, later in Exercise 4 you will perform some transformations to the data. For example, you will extract the sales rank and category from the `salesRank` column and save them into two separate columns.

<a name='3'></a>
## 3 - Raw Data Processing

In this section, you will complete two processing functions that you will later incorporate into the Glue job scripts.

<a name='ex03'></a>
### Exercise 3

In this exercise, you will perform some transformations on the reviews dataset to make the data from the `unixReviewTime` and `helpful` columns more useful for analysis. 

Complete the `process_review()` function with the following transformations:

1. Convert the `unixReviewTime` column to date with the `pd.to_datetime()` function. Remember that this timestamp is defined in seconds (use `s` for the `unit` parameter). Save the result in the column `reviewTime`.
2. Extract the year and month from the `reviewTime`, and save those values in new columns named `year` and `month`. You can apply `.dt.year` and `.dt.month` methods to `raw_df['reviewTime']` to do that. You will later use these columns to partition the processed data in the `S3` bucket.
3. Create a new DataFrame named `df_helpful` based on converting the `helpful` column from the `raw_df` into a list with the `to_list()` method. Set the column names as `helpful` and `totalHelpful`. 
4. With the `pd.concat()` function, concatenate the `raw_df` dataframe with `df_helpful`. From the `raw_df` drop the `helpful` column using the `raw_df.drop()` function, set `axis=1`.

Then, perform the transformations and check the result.

In [ ]:
def process_review(raw_df: pd.DataFrame) -> pd.DataFrame:    
    """Transformations steps for Reviews dataset

    Args:
        raw_df (DataFrame): Raw data loaded in dataframe

    Returns:
        DataFrame: Returned transformed dataframe
    """

    ### START CODE HERE ### (5 lines of code)
    raw_df['reviewTime'] = pd.to_datetime(raw_df['unixReviewTime'], unit='s') # @REPLACE EQUALS pd.None(None['None'], unit='None')
    raw_df['year'] = raw_df['reviewTime'].dt.year # @REPLACE EQUALS None['None'].None.None
    raw_df['month'] = raw_df['reviewTime'].dt.month # @REPLACE EQUALS None['None'].None.None
    
    df_helpful = pd.DataFrame(raw_df['helpful'].to_list(), columns=['helpful', 'totalHelpful']) # @REPLACE EQUALS pd.DataFrame(None['None'].None(), columns=['None', 'None'])
    target_df = pd.concat([raw_df.drop(columns=['helpful']), df_helpful], axis=1) # @REPLACE EQUALS None.None([None.None(columns=['None']), None], axis=1)
    ### END CODE HERE ###
    
    return target_df

transformed_review_sample_df = process_review(raw_df=review_sample_df)
transformed_review_sample_df.head()

<a name='ex04'></a>
### Exercise 4

In this exercise, you will perform some transformations on the reviews metadata dataset to make the data from the `salesRank` column more useful for analysis. You will also remove some of the null values and replace others with an empty string.

Complete the function provided in the next cell with the following steps: 

1. Remove any records that have null values for the `salesRank` column. You can use the `dropna()` method with the parameter `how="any"`. Save the resulting dataframe in the `tmp_df` variable.
2. Create a dataframe named `df_rank`. This dataframe should contain two columns named `'sales_category'` and `'sales_rank'` that come from extracting the key and value respectively from the `'salesRank'` column.
3. Concatenate the `tmp_df` and `df_rank` dataframes and save the result in a new dataframe named `target_df`. You can use function `pd.concat()` to do that.
4. Use the parameter `cols` to select the desired columns in the dataframe `target_df`.
5. From `target_df`, remove any records that have null values for the `asin`, `price` and `sales_rank` columns. Again, use the `dropna()` method with the parameter `how="any"`.
6. Fill the null value of the rest of the dataframe with an empty string `""`. You can use `fillna()` function to do that.

Then, perform the transformations and check the result.

In [ ]:
def process_metadata(raw_df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """Function in charge of the transformation of the raw data of the
    Reviews Metadata.

    Args:
        raw_df (DataFrame): Raw data loaded in dataframe
        cols (list): List of columns to select

    Returns:
        DataFrame: Returned transformed dataframe
    """

    ### START CODE HERE ### (6 lines of code)
    tmp_df = raw_df.dropna(subset=["salesRank"], how="any") # @REPLACE EQUALS None.None(subset=["None"], how="None")
    
    df_rank = pd.DataFrame([{"sales_category": key, "sales_rank": value} for d in tmp_df["salesRank"].tolist() for key, value in d.items()]) # @REPLACE EQUALS pd.DataFrame([{"None": key, "None": value} for d in tmp_df["salesRank"].tolist() for key, value in d.items()])
    
    target_df = pd.concat([tmp_df, df_rank], axis=1) # @REPLACE EQUALS None.None([None, None], axis=1)
    target_df = target_df[cols] # @REPLACE EQUALS target_df[None]
    target_df = target_df.dropna(subset=["asin", "price", "sales_rank"], how="any") # @REPLACE EQUALS target_df.None(subset=["None", "None", "None"], how="None")
    target_df = target_df.fillna("") # @REPLACE EQUALS target_df.None("")
    ### END CODE HERE ###
    
    return target_df

processed_metadata_df = process_metadata(raw_df=metadata_sample_df, 
                                         cols=['asin', 'description', 'title', 'price', 'brand','sales_category','sales_rank']
                                         )
processed_metadata_df.head()


You have explored and performed some transformations on the reviews and metadata sample datasets, and you are now ready to perform those transformations over the complete dataset. This process will be done with AWS Glue.

<a name='4'></a>
## 4 - Transformation with AWS Glue ETL

AWS Glue ETL (Extract, Transform, Load) is a serverless service that simplifies data integration and processing. You briefly used Glue in the labs of course 1. In this lab, you will interact more closely with this service, and in course 4, you will learn more details about its underlying distributed framework (Apache Spark). 

AWS Glue requires a Spark script to perform a job, this script can be coded in Python or Scala. Here you are provided with Python scripts. In these scripts, you will extract the raw data from the provided bucket, transform the data and then save it in the **parquet** format. Parquet is a columnar storage file format commonly used in big data processing frameworks like Apache Hadoop and Apache Spark. Parquet format has several features such as the support of compression algorithms and support of schema evolution. If you want to know more about this format, you can read [this article](https://airbyte.com/data-engineering-resources/parquet-data-format).

<a name='4.1'></a>
### 4.1 - Preparing the Scripts

<a name='ex05'></a>
### Exercise 5

You will now complete the script to transform the Amazon Reviews.

1. Open the file `terraform/assets/de-c3w2-reviews-transform-job.py`. The script reads the data from the JSON file, performs basic transformations, and saves the result in a parquet file.

2. Before completing the incomplete `transform()` function, read through the entire code and comments to get an overview of how the three steps - Extract, Transform, and Load - are implemented.

3. Complete the `transform()` function by copying part of the code you completed in [Exercise 3](#ex03).

4. Save changes to the file.

<a name='ex06'></a>
### Exercise 6

You will now complete the script to transform the Amazon Product Metadata.

1. Open the file `terraform/assets/de-c3w2-metadata-transform-job.py`.

2. Before completing the incomplete `transform()` function, read through the entire code and comments to get an overview of how the three steps - Extract, Transform, and Load - are implemented. 

3. Complete the `transform()` function by copying part of the code that you completed in [Exercise 4](#ex04). 

4. Save changes to the file.

<a name='4.2'></a>
### 4.2 - Creating and Running the Glue Jobs

After completing the scripts, you will now create the resources needed to run the Glue jobs using Terraform. Once you create the resources, you will run the AWS Glue Jobs to ingest the raw data from the source bucket and transform it. You are provided with the Terraform configuration files to create the Glue jobs, your task is to deploy the jobs. For a detailed overview of the provided terraform files, make sure to watch the lab walkthrough video.

4.2.1. Open the `./terraform/glue.tf` file. You will see two different Glue jobs: one for the reviews and another for the product metadata. For now, leave the file as is. Note the use of the `Snappy` compression algorithm for both jobs and the choice of partitioning columns: `year` and `month` for the reviews, and `sales_category` for the product metadata. In the optional part of the lab at the end, you will make changes to the compression algorithm and partitioning columns to perform various experiments.

4.2.2. In the Cloud9 or Jupyter terminal, navigate to the `terraform` folder, initialize Terraform and then deploy the resources by running the following commands:

```bash
cd terraform
terraform init
terraform plan
terraform apply
```

*Note*: After the `terraform apply` command, you will need to type `yes` and press `Enter` to confirm that you want Terraform to apply the changes.

The output will include the name of each of the glue jobs and the IAM role used, as follows:

```bash
glue_role = "Cloud9-de-c3w2lab1-glue-role"
metadata_glue_job = "de-c3w2lab1-metadata-etl-job"
reviews_glue_job = "de-c3w2lab1-reviews-etl-job"
```

These outputs will be used later in the lab, you can save them locally. Note that you will only need the values of the job names (i.e., `de-c3w2lab1-metadata-etl-job` and `de-c3w2lab1-reviews-etl-job`) for the next step. The key names (`metadata_glue_job` and `reviews_glue_job`) are provided for reference but are not used in the commands.

4.2.3. Run each job with the command below, replacing the placeholder `<GLUE-JOB-NAME>` with the job name from the Terraform output (`de-c3w2lab1-reviews-etl-job` or `de-c3w2lab1-metadata-etl-job`):

```bash
aws glue start-job-run --job-name <GLUE-JOB-NAME> | jq -r '.JobRunId' 

4.2.4. You can check the status of each Glue job in the console, or from the terminal by running the following command. Make sure to exchange the `<GLUE-JOB-NAME>` with the job name, and `<JOB-RUN-ID>` with the output from the previous step:

```bash
aws glue get-job-run --job-name <GLUE-JOB-NAME> --run-id <JOB-RUN-ID> --output text --query "JobRun.JobRunState"
```

When each job has a `SUCCEEDED` status, you can continue with the rest of the lab. Take into account that each run of `de-c3w2lab1-metadata-etl-job` can take around 3 minutes while each run of `de-c3w2lab1-reviews-etl-job` can take between 7 to 8 minutes.
- The processed metadata will be stored at: `s3://<BUCKET_NAME>/processed_data/snappy/partition_by_sales_category/toys_metadata/`.
- The processed reviews will be stored at:
`s3://<BUCKET_NAME>/processed_data/snappy/partition_by_year_month/toys_reviews/`.

<a name='5'></a>
## 5 - Data Catalog with Glue Crawler

AWS Glue Crawler is a powerful automated tool offered by AWS Glue for discovering and cataloging metadata about data sources, which enables services like Glue and Athena to query data directly from different sources. By simply pointing the crawler to your data source, whether it's a database or a data lake, it will automatically scan and extract the schema information, data types, and other relevant metadata. This metadata is then organized and stored in a database table in the AWS Glue Data Catalog, providing a centralized repository for understanding and managing your data assets. In this lab, you will create a crawler that will be in charge of populating the Glue Data Catalog with the newly transformed data in S3, you will be able to query the data directly from S3 using Athena in the final part.

Start by checking the databases in the data catalog using AWS SDK for pandas (`awswrangler`):

In [ ]:
databases = wr.catalog.databases()
print(databases)

The Data catalog is empty, let's create a new catalog database labeled as `de-c3w2lab1-aws-reviews` in which the crawler will store the metadata of the processed data.

In [ ]:
DATABASE_NAME = "de-c3w2lab1-aws-reviews"

In [ ]:
if DATABASE_NAME not in databases.values:
    wr.catalog.create_database(DATABASE_NAME)
    print(wr.catalog.databases())
else:
    print(f"Database {DATABASE_NAME} already exists")

<a name='ex07'></a>
### Exercise 7

Complete the code to create a Glue crawler utilizing the `boto3` library:

1. Define the Role parameter with the Glue Job role that you obtained from the `terraform` output (step 4.2.2).
2. Add a description to the crawler (e.g. 'Amazon Reviews for toys').
3. Define the S3 targets with the  paths in your data lake bucket for the `toys_reviews` table (partitioned by year and month and with `snappy` compression) and the `toys_metadata` table (partitioned by sales category with `snappy` compression). You just need to replace `None` with the `BUCKET_NAME` variable.

In [ ]:
glue_client = boto3.client('glue',region_name="us-east-1")
configuration= {"Version": 1.0,"Grouping": {"TableGroupingPolicy": "CombineCompatibleSchemas" }}

response = glue_client.create_crawler(
    Name='de-c3w2lab1-crawler',
    ### START CODE HERE ### (12 lines of code)
    Role='Cloud9-de-c3w2lab1-glue-role', # @REPLACE EQUALS 'None',
    DatabaseName=DATABASE_NAME, # @KEEP
    Description='Amazon Reviews for toys', # @REPLACE EQUALS 'None',
    Targets={ # @KEEP
        'S3Targets': [ # @KEEP
            { # @KEEP
                'Path': f's3://{BUCKET_NAME}/processed_data/snappy/partition_by_year_month/toys_reviews/', # @REPLACE 'Path': f's3://{None}/processed_data/snappy/partition_by_year_month/toys_reviews/',
            },# @KEEP
            { # @KEEP
                'Path': f's3://{BUCKET_NAME}/processed_data/snappy/partition_by_sales_category/toys_metadata/', # @REPLACE 'Path': f's3://{None}/processed_data/snappy/partition_by_sales_category/toys_metadata/',
            } # @KEEP            
        ]} # @KEEP
    
    ### END CODE HERE ###
)

Let's check that the crawler was indeed created, using the `list_crawlers` method of the `boto3` Glue Client. If the creation was successful, you should see the new crawler in the list. 

In [ ]:
response = glue_client.list_crawlers()
print(response['CrawlerNames'])

Start the crawler:

In [ ]:
response = glue_client.start_crawler(
    Name='de-c3w2lab1-crawler'
)
print(response)

The crawler will start looking for data sources in the S3 targets that we have defined, it should take around **3 minutes** for the first run. 

After around 3 minutes, check that the two tables were created for each processed dataset using the following command:

In [ ]:
wr.catalog.tables(database=DATABASE_NAME)

Before proceeding to the last section of the lab where you will use Amazon Athena, make sure that two catalog tables were created.

<a name='6'></a>
## 6 - Data Querying with Athena

Amazon Athena is a serverless, interactive query service provided by AWS, allowing you to analyze data in Amazon S3 using standard SQL. It enables you to quickly and easily query vast amounts of data stored in various formats such as CSV, JSON, Parquet, and more, without needing to set up and manage complex infrastructure. Athena leverages the AWS Glue Data Catalog to access the stored metadata associated with the S3 data. By utilizing this metadata, Athena seamlessly executes queries on the underlying data, streamlining the analytical process and enabling efficient data exploration and analysis.

You will now test some of the queries that the data analyst will run. You will use the `awswrangler` library to run these queries.

*Note*: `awswrangler` uses the `pyarrow` library that has some functions that returns a `FutureWarning` due to deprecation, this doesn't affect what you are trying to do so you will filter out those warnings).

In [ ]:
warnings.simplefilter('ignore', FutureWarning)

Run this test query to retrieve a sample of 5 records from the toys_reviews table.

In [ ]:
sql = "SELECT * FROM toys_reviews LIMIT 5"
df = wr.athena.read_sql_query(sql, database=DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

Next, run this test query to find the top 5 products with the most reviews, return the name of the product and the count of reviews. Ignore the products with an empty title in the metadata table.

In [ ]:
sql = """
SELECT met.title, count(distinct toy.reviewerid) as review_count
FROM toys_metadata met 
LEFT JOIN toys_reviews toy
ON met.asin = toy.asin
WHERE met.title <> ''
GROUP BY met.title
ORDER BY count(distinct toy.reviewerid) DESC
LIMIT 5"""
df = wr.athena.read_sql_query(sql, database=DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

Now run the next test query to find the top 10 products in terms of average rating, but the products should have at least 1000 reviews. Return the title, sales category and the average rating.

In [ ]:
sql = """
SELECT met.title, met.sales_category, avg(toy.overall) as review_avg
FROM toys_metadata met 
LEFT JOIN toys_reviews toy
ON met.asin = toy.asin
GROUP BY met.title, met.sales_category
HAVING count(distinct toy.reviewerid) > 1000
ORDER BY avg(toy.overall) DESC
LIMIT 10"""
df = wr.athena.read_sql_query(sql, database=DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

Next, run this query to determine the average rating for each brand and the number of products they have in the database. Only show the top 10 brands with the highest product counts in the database.

In [ ]:
sql = """
SELECT met.brand, count(distinct met.asin) as product_count, avg(toy.overall) as review_avg
FROM toys_metadata met 
LEFT JOIN toys_reviews toy
ON met.asin = toy.asin
WHERE met.brand <> ''
GROUP BY met.brand
ORDER BY count(distinct toy.asin) DESC
LIMIT 10"""
df = wr.athena.read_sql_query(sql, database=DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

For the last test query, you want to look at 25 random reviews that gave a rating of 5 to a toy car product. Return the product title, description, the review text and overall score. Ignore products that have an empty title, use the LIKE operator to search reviews with toy car in their text.

In [ ]:
sql = """
SELECT met.title, met.description, toy.reviewtext, toy.overall
FROM toys_reviews toy
LEFT JOIN toys_metadata met
ON toy.asin = met.asin
WHERE toy.reviewtext like '%toy car%' and toy.overall = 5.0 and met.title <> '' 
LIMIT 25"""
df = wr.athena.read_sql_query(sql, database=DATABASE_NAME, s3_output=f's3://{BUCKET_NAME}/athena_output/')
df.head()

**Note**:
The next section is optional. If you'd like to try it, first ensure you submit the lab for grading, and then proceed with the experiments. Otherwise, feel free to skip to [the last section](#8). For a summary of the optional section, you can watch the lab walkthrough video.


<a name='7'></a>
## 7 - Optional Experiments: Partitioning and Compression Features of Parquet Format

**Compression**

The Parquet format, commonly used in data lake architectures, supports various [compression codecs](https://parquet.apache.org/docs/file-format/data-pages/compression/) such as Snappy, Gzip, and LZO. These codecs play a crucial role in reducing the storage space needed by Parquet files. However, they come with a trade-off: while they reduce storage costs and optimize resource utilization, they can also impact processing speeds during data ingestion, transformation, and querying. To test this with a practical example, you will store the same transformed data using three different compression options: `UNCOMPRESSED`, `SNAPPY` and `GZIP`: 

- `UNCOMPRESSED`: Data is left uncompressed.
- `SNAPPY`: Snappy is a fast compression/decompression library that works well with Parquet files. It provides good compression ratios and is optimized for speed. Snappy compression is often a good choice for balancing compression efficiency and query performance.
- `GZIP`: Gzip is a widely used compression algorithm that provides high compression ratios. However, it tends to be slower in both compression and decompression compared to Snappy. Gzip compression can achieve higher levels of compression but may result in slower query performance.

**Partitioning**

Partitioning in Parquet is a technique used to better organize data within Parquet files based on partition keys. By partitioning data, Parquet optimizes query performance by reducing the amount of data that needs to be scanned during query execution. 

**Experiments**

To explore the compression features, you will process the product metadata 3 times; in each time, you will choose a different option for the compression algorithm with no partitioning as shown in the left table here. Then you will compare for this data, "uncompressed versus Snappy" and then "Snappy versus Gzip". 

To explore the partitioning features, you will process the review dataset 3 times; in each time, you will choose a different option for the partitioning column, with Snappy for compression. Then you will compare for this data, "partitioning versus no partitioning" and then "partitioning by year and month" versus "partitioning by the product id (asin)".

<img src="images/experiment.png" width="700"/>

<a name='7.1'></a>
### 7.1 - Experiments with the Glue Jobs

Open the file `./terraform/glue.tf` where you will be specifying the parameters for each of the experiments. Remember to save changes by pressing `Ctrl+S` or `Cmd+S` before redeploying each job.

<a name='exp01'></a>
### Metadata - No Partitioning, Uncompressed

Search for the `resource "aws_glue_job" "metadata_etl_job"` terraform resource. In the `default_arguments` parameter, set the following:
- `--compression` to `"uncompressed"` indicating that the parquet files will be saved without any compression,
- `--partition_cols` to `jsonencode([])`, which will be an empty list in the partition columns, no partitioning will be performed. 

Save the changes and follow steps 4.2.2 - 4.2.4. Run only the metadata Glue job. The results of this experiment will be stored at `s3://<BUCKET_NAME>/processed_data/uncompressed/no_partition/toys_metadata/`.

<a name='exp02'></a>
### Metadata - No Partitioning, Snappy

Search for the `resource "aws_glue_job" "metadata_etl_job"` terraform resource. In the `default_arguments` parameter, set 
- `--compression` to `"snappy"` indicating that the parquet files will be compressed using the `SNAPPY` algorithm,
- `--partition_cols` to `jsonencode([])`. 

Again, save the changes and follow steps 4.2.2 - 4.2.4. Run only the metadata Glue job. The results of this experiment will be stored at `s3://<BUCKET_NAME>/processed_data/snappy/no_partition/toys_metadata/`.

<a name='exp03'></a>
### Metadata - No partitioning, Gzip

Search for the `resource "aws_glue_job" "metadata_etl_job"` terraform resource. In the `default_arguments` parameter, set 
- `--compression` to `"gzip"` indicating that the parquet files will be compressed using the `GZIP` algorithm,
- `--partition_cols` to `jsonencode([])`. 

Again, save the changes and follow steps 4.2.2 - 4.2.4. Run only the metadata Glue job. The results of this experiment will be stored at `s3://<BUCKET_NAME>/processed_data/gzip/no_partition/toys_metadata/`.

<a name='exp04'></a>
### Reviews - No partitioning, Snappy

Search for the `resource "aws_glue_job" "reviews_etl_job"` terraform resource. In the `default_arguments` parameter, set 
- `--compression` to `"snappy"`; this will indicate the job that the parquet files will be compressed using the `SNAPPY` algorithm,
- `--partition_cols` to `jsonencode([])`.

Run only the review Glue job. After running the Glue Job, the results of this experiment will be stored at `s3://<BUCKET_NAME>/processed_data/snappy/no_partition/toys_reviews/`.

<a name='exp05'></a>
### Reviews - Partitioning by year and month, Snappy

Here you don't need to run any glue jobs. You've already run this Glue job with Snappy as the compression algorithm and using the year and month columns as the partitioning key. The results are stored at `s3://<BUCKET_NAME>/processed_data/snappy/partition_by_year_month/toys_reviews/`.
The parquet files are partitioned by the value of the two columns, year and month. The first partition is done by `year`and then, each `year` is partitioned into sub-partitions according to the `month`. 


<a name='exp06'></a>
### Reviews - Partitioning by asin, Snappy

Search for the `resource "aws_glue_job" "reviews_etl_job"` terraform resource. In the `default_arguments` parameter, set 
- `--compression` to `"snappy"`; 
- `--partition_cols` to `jsonencode(["asin"])`. 

Create and run only the review Glue Job. Take into account that this experiment is expected to fail due to a **timeout** (It will run for 15 minutes and then stop).

<a name='7.2'></a>
### 7.2 - Experiment Results

Now that your jobs have succeeded, and the processed data of each experiment has been stored in its corresponding locations, in this section you'll analyze the results.

**Compressed vs Uncompressed data**

First, let's compare between uncompressed and compressed data with no partitions using Metadata. Run the following commands to take a look to the size of the stored files. Remember that you can also run these commands in the Cloud9 terminal.

Read the file sizes of Metadata - No partitioning, Uncompressed:

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://{BUCKET_NAME}/processed_data/uncompressed/no_partition/toys_metadata/

Read the file sizes of Metadata - No partitioning, Snappy:

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://{BUCKET_NAME}/processed_data/snappy/no_partition/toys_metadata/

You'll notice that uncompressed files are bigger in size than the compressed ones, by almost a factor of 2. Also, even though you didn't specify a partition key, there are several files instead of only one. Some processing frameworks such as Apache Spark and AWS Glue will by default try to partition your data automatically.

**Snappy vs Gzip**

Now, let's take a look at the results from applying two different compression algorithms.Run the following commands.

Read the file sizes of Metadata - No partitioning, Snappy:

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://{BUCKET_NAME}/processed_data/snappy/no_partition/toys_metadata/

Read the file sizes of Metadata - No partitioning, Gzip:

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://{BUCKET_NAME}/processed_data/gzip/no_partition/toys_metadata/

By comparing the compression algorithms, you can see how `GZIP` has a better compression rate than `SNAPPY`. However, this higher compression rate comes at the cost of being less efficient at querying the data when using `GZIP`.

**Partition keys**

Finally, let's compare the usage of partition keys with the reviews dataset.

**No partition keys vs using partition keys**- Let's look again at the results of Reviews - Partitioning by year and month versus no_partition:

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://{BUCKET_NAME}/processed_data/snappy/no_partition/toys_reviews/

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://{BUCKET_NAME}/processed_data/snappy/partition_by_year_month/toys_reviews/

In the partitioned results, you will find a bunch of folders with a structure similar this:

```bash
.  
└── processed_data/
    └── snappy/
        └── partition_by_year_month/
            └── toys_reviews/
                ├── year = <value-1>/
                |   ├── month = <value-x>
                |   |    ├── run-<TIMESTAMP>-part-block-0-0-r-00000-snappy.parquet
                |   |    ├── run-<TIMESTAMP>-part-block-0-0-r-00001-snappy.parquet
                |   |    ...
                |   |    └── run-<TIMESTAMP>-part-block-0-0-r-<PARTITION-ID>-snappy.parquet
                |   ...
                |   ├── month = <value-z>
                |        ├── run-<TIMESTAMP>-part-block-0-0-r-00000-snappy.parquet
                |        ├── run-<TIMESTAMP>-part-block-0-0-r-00001-snappy.parquet
                |        ...
                |        └── run-<TIMESTAMP>-part-block-0-0-r-<PARTITION-ID>-snappy.parquet
                ...                
                └── year = <value-n>/
                    ├── month = <value-x>
                    |    ├── run-<TIMESTAMP>-part-block-0-0-r-00000-snappy.parquet
                    |    ├── run-<TIMESTAMP>-part-block-0-0-r-00001-snappy.parquet
                    |    ...
                    |    └── run-<TIMESTAMP>-part-block-0-0-r-<PARTITION-ID>-snappy.parquet
                    ...
                    ├── month = <value-z>
                         ├── run-<TIMESTAMP>-part-block-0-0-r-00000-snappy.parquet
                         ├── run-<TIMESTAMP>-part-block-0-0-r-00001-snappy.parquet
                         ...
                         └── run-<TIMESTAMP>-part-block-0-0-r-<PARTITION-ID>-snappy.parquet
```

The folders you see have names made up of the partition key (or column name) and the corresponding partition value. Inside each folder, you'll find one or more parquet files with particular IDs. You can see that the total size is quite similar. 

Partition keys must be set in a way that distributes the data as evenly as possible across the partitions, as you will see in the next experiment.

**Setting appropriate partition keys**:
Now, let's take a look at the results of the last Glue job. The job will run for 15 minutes, and then you will get a timeout error if you check the status of the glue job in the console.  

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://{BUCKET_NAME}/processed_data/snappy/partition_by_year_month/toys_reviews/

In [ ]:
# The execution of this cell may take around 1-2 minutes:
!aws s3 ls --summarize --human-readable --recursive s3://{BUCKET_NAME}/processed_data/snappy/partition_by_asin/toys_reviews/

When comparing the final output of these last two commands, you can see that using the year and month columns as partition keys generated around 556 files with a total size of less than 600MB. On the other hand, using the asin column (which is the identifier for each product) as the partition key can generate thousands of files. However, this job failed due to a timeout, resulting in a smaller total size as not all data was saved. This illustrates one of the issues that may arise when using inappropriate partition keys.

With year and month as partition keys, the number of reviews can be more evenly distributed across dates, resulting in a manageable number of files with sizes which can be later easily processed. However, with the asin partition key, given that some products may have very few reviews and there's a large number of products, the Glue job couldn't save all files within the allotted time.

Having too many small files can have several disadvantages:

* Increased Metadata Overhead: Each Parquet file carries metadata. With numerous small files, managing this metadata can significantly impact performance.
* Higher Storage Costs: Storing many small files can result in higher costs compared to fewer larger files, as cloud storage providers often charge based on the number of objects stored.
* Slower Filesystem Operations: Operations like listing or opening files may become slower with numerous small files, affecting performance of administrative tasks or data processing.
* Suboptimal Data Processing: Some data processing frameworks may be less efficient when handling many small files, incurring overhead in file opening, network communication, and task scheduling.

The key is to balance partitioning data for query performance while avoiding the drawbacks of too many small files. It's recommended to choose a partitioning strategy that aligns with your querying patterns and dataset scale, considering factors like storage costs, filesystem limitations, and processing efficiency.

<a name='8'></a>
## 8 - Clean up

After finishing the experiments and exercises, you will have to delete some of the created resources, let's start with the resources created with terraform. Run the following command inside the `terraform` folder:

```bash
terraform destroy
```

Now, let's delete the AWS Glue crawler and database, run the following cells:

In [ ]:
try:
    response = glue_client.delete_crawler(
        Name='de-c3w2lab1-crawler'
    )
    print("Crawler deleted successfully")
except glue_client.exceptions.EntityNotFoundException:
    print("Crawler does not exist or has already been deleted")


In [ ]:
try:
    wr.catalog.delete_database(name=DATABASE_NAME)
    print(f"Database {DATABASE_NAME} deleted successfully")
except Exception as e:
    print(f"Error deleting database: {e}")

Finally, verify that the resources have been deleted

In [ ]:
response = glue_client.list_crawlers()
print("Remaining crawlers:", response['CrawlerNames'])

databases = wr.catalog.databases()
print("Remaining databases:", databases)

**Well done!** In this lab, you implemented a simple data lake using Amazon S3, AWS Glue ETL, Glue Crawler, and Amazon Athena. You ingested raw data into an S3 bucket, which acted as the storage layer of the data lake. You then used AWS GLUE ETL to transform and prepare the data for analysis. After that, you used the Glue Crawler to catalog the data, and then used Athena to run SQL queries directly against the data lake without the need for complex data movement or transformation. This setup allows for easy and efficient querying of data stored in S3, enabling organizations to derive valuable insights from their data assets.